## Iterable

- `__iter__` 매직 메소드를 구현한 객체를 의미한다.

## Iterator
- `__next__` 매직 메소드를 구현한 객체를 의미한다.

# Iterable object

- list, tuple, set, dict는 원하는 구조의 데이터를 보유하며, `for` 루프를 통해 값을 반복적으로 가져올 수 있다.
- 내장 Internal iterable object 뿐만 아니라, Custom iterable object를 정의할 수 있다.

파이썬의 반복은 Iterable protocol이라는 자체 프로토콜을 사용해 동작한다.  
    
```python
for e in myobject:
    ...
```
    
형태로 객체를 반복할 수 있는지 확인하기 위해 파이썬은 고수준에서 다음 2가지를 차례로 확인한다.  

<br/>

- 객체가 `__next__`나 `__iter__` 이터레이터 메소드 중 하나를 포함하는가?
- 객체가 시퀸스이고 `__len__`과 `__getitem__`을 모두 가졌는가?  
     
폴백 메커니즘(*fallback mechanism*)으로 시퀸스도 반복을 할 수 있으므로, `for`루프에서 반복 가능한 객체를 만드는 방법은  
2가지이다.


# Make Iterable Object

- 객체 반복시, 파이썬은 해당 객체의 `iter()` 함수를 호출한다.
- `iter()` 함수는 해당 객체에 `__iter__` 메소드가 있는지를 확인한다.
- `__iter__` 메소드가 있다면, 이를 실행한다.

```python
from datetime import timedelta

class DateRangeIterable:
    """자체 이터레이터 메소드를 가지고 있는 이터러블"""
    
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._present_day = start_date
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._present_day >= self.end_date:
            raisee StopIteration
        today = self._present_day
        self._present_day += timedelta(days=1)
        return today
```

```bash
>>> for day in DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5)):
...     print(day)
...
2019-01-01
2019-01-02
2019-01-03
2019-01-04
>>>
```

- `for` 루프틑 앞서 만든 객체를 사용해 새로운 반복을 시작한다.
- 새로운 반복을 시작한 객체는 `iter()`함수를 호출하며, 이는 `__iter__` 메소드를 호출한다.
- `__iter__` 메소드는 `self`를 반환하며, 객체 자신이 Iterable임을 알린다.
- 루프의 각 단계에서 `self`의 `next()`함수를 호출한다.
- `next()`함수는 `__next__` 메소드를 호출한다.
- 더 이상 생산할 것이 없을 경우, 파이썬에게 `StopIteration`예외를 발생시켜 알려줘야한다.

```bash
>>> r = DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5))
>>> next(r)
datetime.date(2019, 1, 1)
>>> next(r)
datetime.date(2019, 1, 2)
>>> next(r)
datetime.date(2019, 1, 3)
>>> next(r)
datetime.date(2019, 1, 4)
>>> next(r)
Traceback (most recent call last):
    File "<stdin>", line 1, in <module>
    File ... __next__
        raise StopIteration
StopIteration
>>>
```

- 해당 예제는 실행이 한번 끝나면, 계속 `next()`를 호출할 경우 계속 `StopIteration` 예외가 발생한다.
- 2개 이상의 `for`루프에서는 1개의 `for` 루프만 작동하게 되는 에러가 있다.

```bash
>>> r = DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5))
>>> ", ".join(map(str, r1))
'2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04'
>>> max(r1)
Traceback (most recent call last):
    File "<stdin>", line 1, in <module>
ValueError: max() arg is an empty sequence
>>>
```

- Iterable Object는 이터레이터를 생성하고 이것을 사용해서 반복한다.
- 위의 예제에서ㅓ는 이터레이터가 하나의 이터레이터밖에 없기 때문이다.
- 따라서 이를 해결하기 위해서 매번 새로운 `DateRangeIterable` 인스턴슬를 만들면 된다.
- `__iter__`는 `self`를 반환하지만, 호출될 때마다 새로운 이터레이터를 만들 수 있다.

```python
from datetime import timedelta

class DateRangeIterable:
    """자체 이터레이터 메소드를 가지고 있는 이터러블"""
    
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date        
        
    def __iter__(self):
        current_day = self.start_date
        while current_day < self.end_date:
            yield current_day
            current_day += timedelta(days=1)
        return self    
```

```bash
>>> r = DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5))
>>> ", ".join(map(str, r1))
'2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04'
>>> max(r1)
datetime.date(2019, 1, 4)
>>>
```

- 이러한 형태의 객체를 컨테이너 이터러블(container iterable)이라고 한다.

# Make Sequence Object

- 객체에 `__iter__()` 메소드를 정의하지 않았지만, 반복하기를 원하는 경우도 있다.
- `iter()` 함수는 객체에 `__iter__`가 정의되어 있지 않으면 `__getitem__`을 찾고 없으면 `TypeError`를 발생시킨다.

<br/>

- 시퀸스는 `__len__`과 `__getitem__`을 구현한다.
- 첫 번째 인덱스는 0부터 시작하여 포함된 요소를 한 번에 하나씩 차례로 가져올 수 있어야 한다.
     - `__getitem__`을 올바르게 구현하여 이러한 인덱싱이 가능하도록 신경써야한다.
     
- 이전 섹션 예제는 메모리를 적게 사용한다.
- 하지만, n번째 요소를 얻고 싶으면, n번 반복한다는 단점이 있다.
- 이는 CS분야에서 memory & cpu trade-off problem이다.

<br/>
<br/>

- 이터러블을 사용하면 memory는 적게 사용한다.
- 하지만 n 번째 요소를 얻기 위한 시간복잡도는 O(n)이다.

<br/>

- 시퀸스를 사용하면 더 많은 메모리가 사용된다.
- 특정 요소를 가져오기 위한 인덱싱의 시간복잡도는 O(1)이다.

```python

class DateRangeIterable:

    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date        
        self._range = self._createe_range()

    def _create_ragne(self):
        days = []
        current_day = self.start_date
        while current_day < self.end_date:
            days.append(current_day)
            current_day += timedelta(days=1)
        return days
    
    def __getitem__(self, day_no):
        return self._range[day_no]    
```

```bash
>>> r = DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5))
>>> for day in s1:
...     print(day)
...
2019-01-01
2019-01-02
2019-01-03
2019-01-04
>>> s1[0]
datetime.date(2019, 1, 1)
>>> s1[3]
datetime.date(2019, 1, 4)
>>> s1[-1]
datetime.date(2019, 1, 4)
```

- 음수의 인덱스가 작동하는 이유는 `DateRangeSequence` 객체가 모든 작업을 래핑된 객체인 리스트에게 위임하기 때문이다.
- 두 가지 구현 중 어느 것을 사용할지 결정할 때, 메모리와 CPU 사이의 트레이드오프를 계산해야한다.
- 일반적으로 이터레이션이 더 좋은 선택이지만 (제너레이터는 더욱 바람직하다.) 모든 경우의 요건을 염두에 둬야한다.